In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import re

# display all the attributes
pd.set_option('display.max_columns', None)

In [4]:
dataset1 = pd.read_csv("datasets/dft-road-casualty-statistics-casualty-2022.csv")
dataset2 = pd.read_csv("datasets/dft-road-casualty-statistics-collision-2022.csv")
dataset3 = pd.read_csv("datasets/dft-road-casualty-statistics-vehicle-2022.csv")

# Merge collision data with vehicle data
collision_vehicle_merged = pd.merge(dataset2, dataset3, on="accident_index", how="left")

# Merge the result with casualty data
df = pd.merge(collision_vehicle_merged, dataset1, on="accident_index", how="left")

# Save the merged data to a new CSV file
df


C:\Users\20221803\AppData\Local\Temp\ipykernel_19524\3151081040.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset1 = pd.read_csv("datasets/dft-road-casualty-statistics-casualty-2022.csv")
C:\Users\20221803\AppData\Local\Temp\ipykernel_19524\3151081040.py:2: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset2 = pd.read_csv("datasets/dft-road-casualty-statistics-collision-2022.csv")
C:\Users\20221803\AppData\Local\Temp\ipykernel_19524\3151081040.py:3: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset3 = pd.read_csv("datasets/dft-road-casualty-statistics-vehicle-2022.csv")


,accident_index,accident_year_x,accident_reference_x,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,accident_year_y,accident_reference_y,vehicle_reference_x,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,lsoa_of_driver,accident_year,accident_reference,vehicle_reference_y,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile,lsoa_of_casualty
0,2022010352073,2022,10352073,525199.0,177928.0,-0.198224,51.486454,1,3,2,1,05/01/2022,4,16:40,-1,E09000013,E09000013,3,3218,6,30,6,4,6,0,0,0,4,1,1,0,0,1,1,2,E01001883,2022.0,10352073,1.0,19.0,0.0,9.0,3.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,48.0,8.0,1461.0,2.0,4.0,RENAULT KANGOO,7.0,1.0,E01002687,2022.0,10352073,2.0,1.0,1.0,1.0,17.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,E01001364
1,2022010352073,2022,10352073,525199.0,177928.0,-0.198224,51.486454,1,3,2,1,05/01/2022,4,16:40,-1,E09000013,E09000013,3,3218,6,30,6,4,6,0,0,0,4,1,1,0,0,1,1,2,E01001883,2022.0,10352073,2.0,3.0,0.0,18.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,17.0,4.0,125.0,1.0,1.0,HONDA FSH125,1.0,1.0,E01001364,2022.0,10352073,2.0,1.0,1.0,1.0,17.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,E01001364
2,2022010352573,2022,10352573,546214.0,179866.0,0.105042,51.498830,1,3,2,1,01/01/2022,7,01:17,-1,E09000011,E09000011,3,2016,3,50,6,2,6,0,0,4,4,1,1,0,0,2,1,2,E01033745,2022.0,10352573,1.0,9.0,0.0,18.0,7.0,3.0,0.0,6.0,0.0,0.0,1.0,1.0,1.0,1.0,6.0,2.0,42.0,7.0,-1.0,-1.0,-1.0,-1,5.0,1.0,E01000338,2022.0,10352573,1.0,1.0,1.0,2.0,42.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,5.0,E01000338
3,2022010352573,2022,10352573,546214.0,179866.0,0.105042,51.498830,1,3,2,1,01/01/2022,7,01:17,-1,E09000011,E09000011,3,2016,3,50,6,2,6,0,0,4,4,1,1,0,0,2,1,2,E01033745,2022.0,10352573,2.0,9.0,0.0,13.0,7.0,3.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1,2022.0,10352573,1.0,1.0,1.0,2.0,42.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,5.0,E01000338
4,2022010352575,2022,10352575,551119.0,174789.0,0.173482,51.451924,1,3,2,1,01/01/2022,7,01:15,-1,E09000004,E09000004,3,207,6,30,9,4,6,0,0,0,4,1,1,0,0,1,1,2,E01000378,2022.0,10352575,1.0,9.0,0.0,18.0,4.0,8.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,20.0,4.0,1896.0,2.0,15.0,VOLKSWAGEN GOLF,8.0,1.0,E01000390,2022.0,10352575,1.0,1.0,1.0,1.0,20.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,8.0,E01000390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223235,2022991312498,2022,991312498,127459.0,663067.0,-6.348650,55.783849,99,2,3,1,12/11/2022,7,21:35,-1,S12000035,S12000035,3,847,6,60,0

In [80]:

# Check for unique values and nulls
def check_keys(df, key_columns):
    return {key: (df[key].nunique(), df[key].isnull().sum()) for key in key_columns}

print(check_keys(dataset1, ['accident_index', 'vehicle_reference']))
print(check_keys(dataset3, ['accident_index', 'vehicle_reference']))
print(check_keys(dataset2, ['accident_index']))

{'accident_index': (106004, 0), 'vehicle_reference': (17, 0)}
{'accident_index': (106004, 0), 'vehicle_reference': (24, 0)}
{'accident_index': (106004, 0)}


In [81]:
dataset1

,accident_index,accident_year,accident_reference,vehicle_reference,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile,lsoa_of_casualty
0,2022010352073,2022,10352073,2,1,1,1,17,4,3,0,0,0,0,0,3,1,1,E01001364
1,2022010352573,2022,10352573,1,1,1,2,42,7,3,0,0,0,0,0,9,1,5,E01000338
2,2022010352575,2022,10352575,1,1,1,1,20,4,3,0,0,0,0,0,9,1,8,E01000390
3,2022010352578,2022,10352578,1,1,1,1,46,8,3,0,0,0,0,0,9,1,3,E01004666
4,2022010352578,2022,10352578,1,2,2,1,45,7,3,0,0,1,0,0,9,1,3,E01004666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135475,2022991311627,2022,991311627,1,1,3,2,69,10,2,4,1,0,0,2,0,-1,-1,-1
135476,2022991312498,2022,991312498,1,1,1,1,36,7,2,0,0,0,0,0,19,3,6,-1
135477,2022991315177,2022,991315177,1,1,3,1,21,5,3,10,9,0,0,2,0,-1,-1,-1
135478,2022991321308,2022,991321308,1,1,3,2,28,6,2,8,5,0,0,2,0,-1,-1,-1


In [82]:
dataset2

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,2022010352073,2022,10352073,525199.0,177928.0,-0.198224,51.486454,1,3,2,1,05/01/2022,4,16:40,-1,E09000013,E09000013,3,3218,6,30,6,4,6,0,0,0,4,1,1,0,0,1,1,2,E01001883
1,2022010352573,2022,10352573,546214.0,179866.0,0.105042,51.498830,1,3,2,1,01/01/2022,7,01:17,-1,E09000011,E09000011,3,2016,3,50,6,2,6,0,0,4,4,1,1,0,0,2,1,2,E01033745
2,2022010352575,2022,10352575,551119.0,174789.0,0.173482,51.451924,1,3,2,1,01/01/2022,7,01:15,-1,E09000004,E09000004,3,207,6,30,9,4,6,0,0,0,4,1,1,0,0,1,1,2,E01000378
3,2022010352578,2022,10352578,528889.0,192230.0,-0.139873,51.614153,1,3,2,2,01/01/2022,7,02:24,-1,E09000010,E09000010,6,0,6,30,0,-1,0,-1,0,0,4,1,1,0,0,1,1,2,E01001529
4,2022010352580,2022,10352580,539773.0,190404.0,0.016495,51.595151,1,3,4,3,01/01/2022,7,02:30,-1,E09000026,E09000026,3,406,3,50,5,4,6,0,0,0,4,1,1,0,0,1,1,2,E01003673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105999,2022991311627,2022,991311627,235016.0,625551.0,-4.613246,55.495815,99,2,1,1,24/12/2022,7,15:00,-1,S12000028,S12000028,3,79,6,30,6,2,6,0,0,1,1,1,1,0,0,1,2,-1,-1
106000,2022991312498,2022,991312498,127459.0,663067.0,-6.348650,55.783849,99,2,3,1,12/11/2022,7,21:35,-1,S12000035,S12000035,3,847,6,60,0,-1,0,-1,0,0,6,1,1,0,0,2,1,-1,-1
106001,2022991315177,2022,991315177,254396.0,663549.0,-4.326930,55.843114,99,3,1,1,01/07/2022,6,11:44,-1,S12000049,S12000049,6,0,6,30,6,2,6,0,0,5,1,1,1,0,0,1,2,-1,-1
106002,2022991321308,2022,991321308,325409.0,677054.0,-3.196963,55.980648,99,2,1,1,02/12/2022,6,16:45,-1,S12000036,S12000036,3,901,6,30,3,4,3,901,0,8,4,1,1,0,0,1,1,-1,-1


In [83]:
dataset3

,accident_index,accident_year,accident_reference,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,lsoa_of_driver
0,2022010352073,2022,10352073,1,19,0,9,3,8,0,1,0,0,0,0,1,2,2,1,48,8,1461,2,4,RENAULT KANGOO,7,1,E01002687
1,2022010352073,2022,10352073,2,3,0,18,1,5,0,1,0,0,0,0,1,1,2,1,17,4,125,1,1,HONDA FSH125,1,1,E01001364
2,2022010352573,2022,10352573,1,9,0,18,7,3,0,6,0,0,1,1,1,1,6,2,42,7,-1,-1,-1,-1,5,1,E01000338
3,2022010352573,2022,10352573,2,9,0,13,7,3,0,6,0,0,0,0,1,1,6,3,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,2022010352575,2022,10352575,1,9,0,18,4,8,0,2,0,0,0,0,1,1,6,1,20,4,1896,2,15,VOLKSWAGEN GOLF,8,1,E01000390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193540,2022991312498,2022,991312498,2,19,0,18,8,1,0,0,0,0,0,0,1,1,5,1,43,7,1968,2,0,VOLKSWAGEN CRAFTER,6,3,-1
193541,2022991312498,2022,991312498,3,9,0,2,8,1,0,0,0,0,0,0,1,1,5,1,62,9,1229,1,12,VAUXHALL CORSA,6,3,-1
193542,2022991315177,2022,991315177,1,9,0,5,8,4,0,2,0,0,0,0,4,1,6,1,61,9,1497,8,10,TOYOTA YARIS,3,1,-1
193543,2022991321308,2022,991321308,1,9,0,18,3,7,0,1,-1,-1,-1,0,1,1,5,1,77,11,1794,1,17,TOYOTA AVENSIS,10,1,-1


In [84]:
# df = pd.read_csv(r'C:\Users\20221343\OneDrive - TU Eindhoven\Desktop\Cleaned-CollisionData2022.csv',low_memory=False,error_bad_lines = False)
# df

In [85]:
df

,accident_index,accident_year_x,accident_reference_x,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,accident_year_y,accident_reference_y,vehicle_reference_x,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,lsoa_of_driver,accident_year,accident_reference,vehicle_reference_y,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile,lsoa_of_casualty
0,2022010352073,2022,10352073,525199.0,177928.0,-0.198224,51.486454,1,3,2,1,05/01/2022,4,16:40,-1,E09000013,E09000013,3,3218,6,30,6,4,6,0,0,0,4,1,1,0,0,1,1,2,E01001883,2022.0,10352073,1.0,19.0,0.0,9.0,3.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,48.0,8.0,1461.0,2.0,4.0,RENAULT KANGOO,7.0,1.0,E01002687,2022.0,10352073,2.0,1.0,1.0,1.0,17.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,E01001364
1,2022010352073,2022,10352073,525199.0,177928.0,-0.198224,51.486454,1,3,2,1,05/01/2022,4,16:40,-1,E09000013,E09000013,3,3218,6,30,6,4,6,0,0,0,4,1,1,0,0,1,1,2,E01001883,2022.0,10352073,2.0,3.0,0.0,18.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,17.0,4.0,125.0,1.0,1.0,HONDA FSH125,1.0,1.0,E01001364,2022.0,10352073,2.0,1.0,1.0,1.0,17.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,E01001364
2,2022010352573,2022,10352573,546214.0,179866.0,0.105042,51.498830,1,3,2,1,01/01/2022,7,01:17,-1,E09000011,E09000011,3,2016,3,50,6,2,6,0,0,4,4,1,1,0,0,2,1,2,E01033745,2022.0,10352573,1.0,9.0,0.0,18.0,7.0,3.0,0.0,6.0,0.0,0.0,1.0,1.0,1.0,1.0,6.0,2.0,42.0,7.0,-1.0,-1.0,-1.0,-1,5.0,1.0,E01000338,2022.0,10352573,1.0,1.0,1.0,2.0,42.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,5.0,E01000338
3,2022010352573,2022,10352573,546214.0,179866.0,0.105042,51.498830,1,3,2,1,01/01/2022,7,01:17,-1,E09000011,E09000011,3,2016,3,50,6,2,6,0,0,4,4,1,1,0,0,2,1,2,E01033745,2022.0,10352573,2.0,9.0,0.0,13.0,7.0,3.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1,2022.0,10352573,1.0,1.0,1.0,2.0,42.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,5.0,E01000338
4,2022010352575,2022,10352575,551119.0,174789.0,0.173482,51.451924,1,3,2,1,01/01/2022,7,01:15,-1,E09000004,E09000004,3,207,6,30,9,4,6,0,0,0,4,1,1,0,0,1,1,2,E01000378,2022.0,10352575,1.0,9.0,0.0,18.0,4.0,8.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,20.0,4.0,1896.0,2.0,15.0,VOLKSWAGEN GOLF,8.0,1.0,E01000390,2022.0,10352575,1.0,1.0,1.0,1.0,20.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,8.0,E01000390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223235,2022991312498,2022,991312498,127459.0,663067.0,-6.348650,55.783849,99,2,3,1,12/11/2022,7,21:35,-1,S12000035,S12000035,3,847,6,60,0

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223240 entries, 0 to 223239
Data columns (total 81 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   accident_index                               223240 non-null  object 
 1   accident_year_x                              223240 non-null  int64  
 2   accident_reference_x                         223240 non-null  object 
 3   location_easting_osgr                        223198 non-null  float64
 4   location_northing_osgr                       223198 non-null  float64
 5   longitude                                    223198 non-null  float64
 6   latitude                                     223198 non-null  float64
 7   police_force                                 223240 non-null  int64  
 8   accident_severity                            223240 non-null  int64  
 9   number_of_vehicles                           223240 non-nul

In [87]:
#df.replace('-1', pd.NA, inplace=True)
# df = df.dropna()


In [88]:
df.isna().sum()

accident_index                            0
accident_year_x                           0
accident_reference_x                      0
location_easting_osgr                    42
location_northing_osgr                   42
                                      ...  
pedestrian_road_maintenance_worker    40326
casualty_type                         40326
casualty_home_area_type               40326
casualty_imd_decile                   40326
lsoa_of_casualty                      40326
Length: 81, dtype: int64

In [89]:
#df.dropna(inplace=True)
#df.isna().sum()

In [90]:
df["police_force"].replace({
    1: "Metropolitan Police",
    3: "Cumbria",
    4: "Lancashire",
    5: "Merseyside",
    6: "Greater Manchester",
    7: "Cheshire",
    10: "Northumbria",
    11: "Durham",
    12: "North Yorkshire",
    13: "West Yorkshire",
    14: "South Yorkshire",
    16: "Humberside",
    17: "Cleveland",
    20: "West Midlands",
    21: "Staffordshire",
    22: "West Mercia",
    23: "Warwickshire",
    30: "Derbyshire",
    31: "Nottinghamshire",
    32: "Lincolnshire",
    33: "Leicestershire",
    34: "Northamptonshire",
    35: "Cambridgeshire",
    36: "Norfolk",
    37: "Suffolk",
    40: "Bedfordshire",
    41: "Hertfordshire",
    42: "Essex",
    43: "Thames Valley",
    44: "Hampshire",
    45: "Surrey",
    46: "Kent",
    47: "Sussex",
    48: "City of London",
    50: "Devon and Cornwall",
    52: "Avon and Somerset",
    53: "Gloucestershire",
    54: "Wiltshire",
    55: "Dorset",
    60: "North Wales",
    61: "Gwent",
    62: "South Wales",
    63: "Dyfed-Powys",
    91: "Northern",
    92: "Grampian",
    93: "Tayside",
    94: "Fife",
    95: "Lothian and Borders",
    96: "Central",
    97: "Strathclyde",
    98: "Dumfries and Galloway",
    99: "Police Scotland"
}, inplace=True)


In [91]:
df["accident_severity"].replace({
    1: "Fatal",
    2: "Serious",
    3: "Slight"
}, inplace=True)


In [92]:
df["day_of_week"].replace({
    1: "Sunday",
    2: "Monday",
    3: "Tuesday",
    4: "Wednesday",
    5: "Thursday",
    6: "Friday",
    7: "Saturday"
}, inplace=True)


In [93]:
df["local_authority_district"].replace({
    1: "Westminster",
    2: "Camden",
    3: "Islington",
    4: "Hackney",
    5: "Tower Hamlets",
    6: "Greenwich",
    7: "Lewisham",
    8: "Southwark",
    9: "Lambeth",
    10: "Wandsworth",
    11: "Hammersmith and Fulham",
    12: "Kensington and Chelsea",
    13: "Waltham Forest",
    14: "Redbridge",
    15: "Havering",
    16: "Barking and Dagenham",
    17: "Newham",
    18: "Bexley",
    19: "Bromley",
    20: "Croydon",
    21: "Sutton",
    22: "Merton",
    23: "Kingston upon Thames",
    24: "Richmond upon Thames",
    25: "Hounslow",
    26: "Hillingdon",
    27: "Ealing",
    28: "Brent",
    29: "Harrow",
    30: "Barnet",
    31: "Haringey",
    32: "Enfield",
    33: "Hertsmere",
    38: "Epsom and Ewell",
    40: "Spelthorne",
    57: "London Airport (Heathrow)",
    60: "Allerdale",
    61: "Barrow-in-Furness",
    62: "Carlisle",
    63: "Copeland",
    64: "Eden",
    65: "South Lakeland",
    70: "Blackburn with Darwen",
    71: "Blackpool",
    72: "Burnley",
    73: "Chorley",
    74: "Fylde",
    75: "Hyndburn",
    76: "Lancaster",
    77: "Pendle",
    79: "Preston",
    80: "Ribble Valley",
    82: "Rossendale",
    83: "South Ribble",
    84: "West Lancashire",
    85: "Wyre",
    90: "Knowsley",
    91: "Liverpool",
    92: "St. Helens",
    93: "Sefton",
    95: "Wirral",
    100: "Bolton",
    101: "Bury",
    102: "Manchester",
    104: "Oldham",
    106: "Rochdale",
    107: "Salford",
    109: "Stockport",
    110: "Tameside",
    112: "Trafford",
    114: "Wigan",
    120: "Chester",
    121: "Congleton",
    122: "Crewe and Nantwich",
    123: "Ellesmere Port and Neston",
    124: "Halton",
    126: "Macclesfield",
    127: "Vale Royal",
    128: "Warrington",
    129: "Cheshire East",
    130: "Cheshire West and Chester",
    139: "Northumberland",
    140: "Alnwick",
    141: "Berwick-upon-Tweed",
    142: "Blyth Valley",
    143: "Castle Morpeth",
    144: "Tynedale",
    145: "Wansbeck",
    146: "Gateshead",
    147: "Newcastle upon Tyne",
    148: "North Tyneside",
    149: "South Tyneside",
    150: "Sunderland",
    160: "Chester-le-Street",
    161: "Darlington",
    162: "Derwentside",
    163: "Durham",
    164: "Easington",
    165: "Sedgefield",
    166: "Teesdale",
    168: "Wear Valley",
    169: "County Durham",
    180: "Craven",
    181: "Hambleton",
    182: "Harrogate",
    184: "Richmondshire",
    185: "Ryedale",
    186: "Scarborough",
    187: "Selby",
    189: "York",
    200: "Bradford",
    202: "Calderdale",
    203: "Kirklees",
    204: "Leeds",
    206: "Wakefield",
    210: "Barnsley",
    211: "Doncaster",
    213: "Rotherham",
    215: "Sheffield",
    228: "Kingston upon Hull, City of",
    231: "East Riding of Yorkshire",
    232: "North Lincolnshire",
    233: "North East Lincolnshire",
    240: "Hartlepool",
    241: "Redcar and Cleveland",
    243: "Middlesbrough",
    245: "Stockton-on-Tees",
    250: "Cannock Chase",
    251: "East Staffordshire",
    252: "Lichfield",
    253: "Newcastle-under-Lyme",
    254: "South Staffordshire",
    255: "Stafford",
    256: "Staffordshire Moorlands",
    257: "Stoke-on-Trent",
    258: "Tamworth",
    270: "Bromsgrove",
    273: "Malvern Hills",
    274: "Redditch",
    276: "Worcester",
    277: "Wychavon",
    278: "Wyre Forest",
    279: "Bridgnorth",
    280: "North Shropshire",
    281: "Oswestry",
    282: "Shrewsbury and Atcham",
    283: "South Shropshire",
    284: "Telford and Wrekin",
    285: "Herefordshire, County of",
    286: "Shropshire",
    290: "North Warwick", 
    291: "Nuneaton and Bedworth",
    292: "Rugby",
    293: "Stratford-upon-Avon",
    294: "Warwick",
    300: "Birmingham",
    302: "Coventry",
    303: "Dudley",
    305: "Sandwell",
    306: "Solihull",
    307: "Walsall",
    309: "Wolverhampton",
    320: "Amber Valley",
    321: "Bolsover",
    322: "Chesterfield",
    323: "Derby",
    324: "Erewash",
    325: "High Peak",
    327: "North East Derbyshire",
    328: "South Derbyshire",
    329: "Derbyshire Dales",
    340: "Ashfield",
    341: "Bassetlaw",
    342: "Broxtowe",
    343: "Gedling",
    344: "Mansfield",
    345: "Newark and Sherwood",
    346: "Nottingham",
    347: "Rushcliffe",
    350: "Boston",
    351: "East Lindsey",
    352: "Lincoln",
    353: "North Kesteven",
    354: "South Holland",
    355: "South Kesteven",
    356: "West Lindsey",
    360: "Blaby",
    361: "Hinckley and Bosworth",
    362: "Charnwood",
    363: "Harborough",
    364: "Leicester",
    365: "Melton",
    366: "North West Leicestershire",
    367: "Oadby and Wigston",
    368: "Rutland",
    380: "Corby",
    381: "Daventry",
    382: "East Northamptonshire",
    383: "Kettering",
    384: "Northampton",
    385: "South Northamptonshire",
    386: "Wellingborough",
    390: "Cambridge",
    391: "East Cambridgeshire",
    392: "Fenland",
    393: "Huntingdonshire",
    394: "Peterborough",
    395: "South Cambridgeshire",
    400: "Breckland",
    401: "Broadland",
    402: "Great Yarmouth",
    404: "Norwich",
    405: "North Norfolk",
    406: "South Norfolk",
    407: "King's Lynn and West Norfolk",
    410: "Babergh",
    411: "Forest Heath",
    412: "Ipswich",
    413: "Mid Suffolk",
    414: "St. Edmundsbury",
    415: "Suffolk Coastal",
    416: "Waveney",
    420: "Bedford",
    421: "Luton",
    422: "Mid Bedfordshire",
    423: "South Bedfordshire",
    424: "Central Bedfordshire",
    430: "Broxbourne",
    431: "Dacorum",
    432: "East Hertfordshire",
    433: "North Hertfordshire",
    434: "St. Albans",
    435: "Stevenage",
    436: "Three Rivers",
    437: "Watford",
    438: "Welwyn Hatfield",
    450: "Basildon",
    451: "Braintree",
    452: "Brentwood",
    453: "Castle Point",
    454: "Chelmsford",
    455: "Colchester",
    456: "Epping Forest",
    457: "Harlow",
    458: "Maldon",
    459: "Rochford",
    460: "Southend-on-Sea",
    461: "Tendring",
    462: "Thurrock",
    463: "Uttlesford",
    470: "Bracknell Forest",
    471: "West Berkshire",
    472: "Reading",
    473: "Slough",
    474: "Windsor and Maidenhead",
    475: "Wokingham",
    476: "Aylesbury Vale",
    477: "South Bucks",
    478: "Chiltern",
    479: "Milton Keynes",
    480: "Wycombe",
    481: "Cherwell",
    482: "Oxford",
    483: "Vale of White Horse",
    484: "South Oxfordshire",
    485: "West Oxfordshire",
    490: "Basingstoke and Deane",
    491: "Eastleigh",
    492: "Fareham",
    493: "Gosport",
    494: "Hart",
    495: "Havant",
    496: "New Forest",
    497: "East Hampshire",
    498: "Portsmouth",
    499: "Rushmoor",
    500: "Southampton",
    501: "Test Valley",
    502: "Winchester",
    505: "Isle of Wight",
    510: "Elmbridge",
    511: "Guildford",
    512: "Mole Valley",
    513: "Reigate and Banstead",
    514: "Runnymede",
    515: "Surrey Heath",
    516: "Tandridge",
    517: "Waverley",
    518: "Woking",
    530: "Ashford",
    531: "Canterbury",
    532: "Dartford",
    533: "Dover",
    535: "Gravesham",
    536: "Maidstone",
    538: "Sevenoaks",
    539: "Shepway",
    540: "Swale",
    541: "Thanet",
    542: "Tonbridge and Malling",
    543: "Tunbridge Wells",
    544: "Medway",
    551: "Eastbourne",
    552: "Hastings",
    554: "Lewes",
    555: "Rother",
    556: "Wealden",
    557: "Adur",
    558: "Arun",
    559: "Chichester",
    560: "Crawley",
    562: "Horsham",
    563: "Mid Sussex",
    564: "Worthing",
    565: "Brighton and Hove",
    570: "City of London",
    580: "East Devon",
    581: "Exeter",
    582: "North Devon",
    583: "Plymouth",
    584: "South Hams",
    585: "Teignbridge",
    586: "Mid Devon",
    587: "Torbay",
    588: "Torridge",
    589: "West Devon",
    590: "Caradon",
    591: "Carrick",
    592: "Kerrier",
    593: "North Cornwall",
    594: "Penwith",
    595: "Restormel",
    596: "Cornwall",
    601: "Bristol, City of",
    605: "North Somerset",
    606: "Mendip",
    607: "Sedgemoor",
    608: "Taunton Deane",
    609: "West Somerset",
    610: "South Somerset",
    611: "Bath and North East Somerset",
    612: "South Gloucestershire",
    620: "Cheltenham",
    621: "Cotswold",
    622: "Forest of Dean",
    623: "Gloucester",
    624: "Stroud",
    625: "Tewkesbury",
    630: "Kennet",
    631: "North Wiltshire",
    632: "Salisbury",
    633: "Swindon",
    634: "West Wiltshire",
    635: "Wiltshire",
    640: "Bournemouth",
    641: "Christchurch",
    642: "North Dorset",
    643: "Poole",
    644: "Purbeck",
    645: "West Dorset",
    646: "Weymouth and Portland",
    647: "East Dorset",
    720: "Isle of Anglesey",
    721: "Conwy",
    722: "Gwynedd",
    723: "Denbighshire",
    724: "Flintshire",
    725: "Wrexham",
    730: "Blaenau Gwent",
    731: "Caerphilly",
    732: "Monmouthshire",
    733: "Newport",
    734: "Torfaen",
    740: "Bridgend",
    741: "Cardiff",
    742: "Merthyr Tydfil",
    743: "Neath Port Talbot",
    744: "Rhondda, Cynon, Taff",
    745: "Swansea",
    746: "The Vale of Glamorgan",
    750: "Ceredigion",
    751: "Carmarthenshire",
    752: "Pembrokeshire",
    753: "Powys",
    910: "Aberdeen City",
    911: "Aberdeenshire",
    912: "Angus",
    913: "Argyll and Bute",
    914: "Scottish Borders",
    915: "Clackmannanshire",
    916: "West Dunbartonshire",
    917: "Dumfries and Galloway",
    918: "Dundee City",
    919: "East Ayrshire",
    920: "East Dunbartonshire",
    921: "East Lothian",
    922: "East Renfrewshire",
    923: "Edinburgh, City of",
    924: "Falkirk",
    925: "Fife",
    926: "Glasgow City",
    927: "Highland",
    928: "Inverclyde",
    929: "Midlothian",
    930: "Moray",
    931: "North Ayrshire",
    932: "North Lanarkshire",
    933: "Orkney Islands",
    934: "Perth and Kinross",
    935: "Renfrewshire",
    936: "Shetland Islands",
    937: "South Ayrshire",
    938: "South Lanarkshire",
    939: "Stirling",
    940: "West Lothian",
    941: "Western Isles"
})
    


0        -1
1        -1
2        -1
3        -1
4        -1
         ..
223235   -1
223236   -1
223237   -1
223238   -1
223239   -1
Name: local_authority_district, Length: 223240, dtype: int64

In [94]:
df["local_authority_ons_district"].replace({
    "E06000001": "Hartlepool",
    "E06000002": "Middlesbrough",
    "E06000003": "Redcar and Cleveland",
    "E06000004": "Stockton-on-Tees",
    "E06000005": "Darlington",
    "E06000006": "Halton",
    "E06000007": "Warrington",
    "E06000008": "Blackburn with Darwen",
    "E06000009": "Blackpool",
    "E06000010": "Kingston upon Hull, City of",
    "E06000011": "East Riding of Yorkshire",
    "E06000012": "North East Lincolnshire",
    "E06000013": "North Lincolnshire",
    "E06000014": "York",
    "E06000015": "Derby",
    "E06000016": "Leicester",
    "E06000017": "Rutland",
    "E06000018": "Nottingham",
    "E06000019": "Herefordshire, County of",
    "E06000020": "Telford and Wrekin",
    "E06000021": "Stoke-on-Trent",
    "E06000001": "Hartlepool",
    "E06000002": "Middlesbrough",
    "E06000003": "Redcar and Cleveland",
    "E06000004": "Stockton-on-Tees",
    "E06000005": "Darlington",
    "E06000006": "Halton",
    "E06000007": "Warrington",
    "E06000008": "Blackburn with Darwen",
    "E06000009": "Blackpool",
    "E06000010": "Kingston upon Hull, City of",
    "E06000011": "East Riding of Yorkshire",
    "E06000012": "North East Lincolnshire",
    "E06000013": "North Lincolnshire",
    "E06000014": "York",
    "E06000015": "Derby",
    "E06000016": "Leicester",
    "E06000017": "Rutland",
    "E06000018": "Nottingham",
    "E06000019": "Herefordshire, County of",
    "E06000020": "Telford and Wrekin",
    "E06000021": "Stoke-on-Trent",
    "E06000022": "Bath and North East Somerset",
    "E06000023": "Bristol, City of",
    "E06000024": "North Somerset",
    "E06000025": "South Gloucestershire",
    "E06000026": "Plymouth",
    "E06000027": "Torbay",
    "E06000028": "Bournemouth",
    "E06000029": "Poole",
    "E06000030": "Swindon",
    "E06000031": "Peterborough",
    "E06000032": "Luton",
    "E06000033": "Southend-on-Sea",
    "E06000034": "Thurrock",
    "E06000035": "Medway",
    "E06000036": "Bracknell Forest",
    "E06000037": "West Berkshire",
    "E06000038": "Reading",
    "E06000039": "Slough",
    "E06000040": "Windsor and Maidenhead",
    "E06000041": "Wokingham",
    "E06000042": "Milton Keynes",
    "E06000043": "Brighton and Hove",
    "E06000044": "Portsmouth",
    "E06000045": "Southampton",
    "E06000046": "Isle of Wight",
    "E06000047": "County Durham",
    "E06000048": "Northumberland",
    "E06000049": "Cheshire East",
    "E06000050": "Cheshire West and Chester",
    "E06000051": "Shropshire",
    "E06000052": "Cornwall",
    "E06000053": "Isles of Scilly",
    "E06000054": "Wiltshire",
    "E06000055": "Bedford",
    "E06000056": "Central Bedfordshire",
    "E07000004": "Aylesbury Vale",
    "E07000005": "Chiltern",
    "E07000006": "South Bucks",
    "E07000007": "Wycombe",
    "E07000008": "Cambridge",
    "E07000009": "East Cambridgeshire",
    "E07000010": "Fenland",
    "E07000011": "Huntingdonshire",
    "E07000012": "South Cambridgeshire",
    "E07000026": "Allerdale",
    "E07000027": "Barrow-in-Furness",
    "E07000028": "Carlisle",
    "E07000029": "Copeland",
    "E07000030": "Eden",
    "E07000031": "South Lakeland",
    "E07000032": "Amber Valley",
    "E07000033": "Bolsover",
    "E07000034": "Chesterfield",
    "E07000035": "Derbyshire Dales",
    "E07000036": "Erewash",
    "E07000037": "High Peak",
    "E07000038": "North East Derbyshire",
    "E07000039": "South Derbyshire",
    "E07000040": "East Devon",
    "E07000041": "Exeter",
    "E07000042": "Mid Devon",
    "E07000043": "North Devon",
    "E07000044": "South Hams",
    "E07000045": "Teignbridge",
    "E07000046": "Torridge",
    "E07000047": "West Devon",
    "E07000048": "Christchurch",
    "E07000049": "East Dorset",
    "E07000050": "North Dorset",
    "E07000051": "Purbeck",
    "E07000052": "West Dorset",
    "E07000053": "Weymouth and Portland",
    "E07000061": "Eastbourne",
    "E07000062": "Hastings",
    "E07000063": "Lewes",
    "E07000064": "Rother",
    "E07000065": "Wealden",
    "E07000066": "Basildon",
    "E07000067": "Braintree",
    "E07000068": "Brentwood",
    "E07000069": "Castle Point",
    "E07000070": "Chelmsford",
    "E07000071": "Colchester",
    "E07000072": "Epping Forest",
    "E07000073": "Harlow",
    "E07000074": "Maldon",
    "E07000075": "Rochford",
    "E07000076": "Tendring",
    "E07000077": "Uttlesford",
    "E07000078": "Cheltenham",
    "E07000079": "Cotswold",
    "E07000080": "Forest of Dean",
    "E07000081": "Gloucester",
    "E07000082": "Stroud",
    "E07000083": "Tewkesbury",
    "E07000084": "Basingstoke and Deane",
    "E07000085": "East Hampshire",
    "E07000086": "Eastleigh",
    "E07000087": "Fareham",
    "E07000088": "Gosport",
    "E07000089": "Hart",
    "E07000090": "Havant",
    "E07000091": "New Forest",
    "E07000092": "Rushmoor",
    "E07000093": "Test Valley",
    "E07000094": "Winchester",
    "E07000095": "Broxbourne",
    "E07000096": "Dacorum",
    "E07000097": "East Hertfordshire",
    "E07000098": "Hertsmere",
    "E07000099": "North Hertfordshire",
    "E07000100": "St Albans",
    "E07000101": "Stevenage",
    "E07000102": "Three Rivers",
    "E07000103": "Watford",
    "E07000104": "Welwyn Hatfield",
    "E07000105": "Ashford",
    "E07000106": "Canterbury",
    "E07000107": "Dartford",
    "E07000108": "Dover",
    "E07000109": "Gravesham",
    "E07000110": "Maidstone",
    "E07000111": "Sevenoaks",
    "E07000112": "Shepway",
    "E07000113": "Swale",
    "E07000114": "Thanet",
    "E07000115": "Tonbridge and Malling",
    "E07000116": "Tunbridge Wells",
    "E07000117": "Burnley",
    "E07000118": "Chorley",
    "E07000119": "Fylde",
    "E07000120": "Hyndburn",
    "E07000121": "Lancaster",
    "E07000122": "Pendle",
    "E07000123": "Preston",
    "E07000124": "Ribble Valley",
    "E07000125": "Rossendale",
    "E07000126": "South Ribble",
    "E07000127": "West Lancashire",
    "E07000128": "Wyre",
    "E07000129": "Blaby",
    "E07000130": "Charnwood",
    "E07000131": "Harborough",
    "E07000132": "Hinckley and Bosworth",
    "E07000133": "Melton",
    "E07000134": "North West Leicestershire",
    "E07000135": "Oadby and Wigston",
    "E07000136": "Boston",
    "E07000137": "East Lindsey",
    "E07000138": "Lincoln",
    "E07000139": "North Kesteven",
    "E07000140": "South Holland",
    "E07000141": "South Kesteven",
    "E07000142": "West Lindsey",
    "E07000143": "Breckland",
    "E07000144": "Broadland",
    "E07000145": "Great Yarmouth",
    "E07000146": "King's Lynn and West Norfolk",
    "E07000147": "North Norfolk",
    "E07000148": "Norwich",
    "E07000149": "South Norfolk",
    "E07000150": "Corby",
    "E07000151": "Daventry",
    "E07000152": "East Northamptonshire",
    "E07000153": "Kettering",
    "E07000154": "Northampton",
    "E07000155": "South Northamptonshire",
    "E07000156": "Wellingborough",
    "E07000163": "Craven",
    "E07000164": "Hambleton",
    "E07000165": "Harrogate",
    "E07000166": "Richmondshire",
    "E07000167": "Ryedale",
    "E07000168": "Scarborough",
     "E07000169": "Selby",
    "E07000170": "Ashfield",
    "E07000171": "Bassetlaw",
    "E07000172": "Broxtowe",
    "E07000173": "Gedling",
    "E07000174": "Mansfield",
    "E07000175": "Newark and Sherwood",
    "E07000176": "Rushcliffe",
    "E07000177": "Cherwell",
    "E07000178": "Oxford",
    "E07000179": "South Oxfordshire",
    "E07000180": "Vale of White Horse",
    "E07000181": "West Oxfordshire",
    "E07000187": "Mendip",
    "E07000188": "Sedgemoor",
    "E07000189": "South Somerset",
    "E07000190": "Taunton Deane",
    "E07000191": "West Somerset",
    "E07000192": "Cannock Chase",
    "E07000193": "East Staffordshire",
    "E07000194": "Lichfield",
    "E07000195": "Newcastle-under-Lyme",
    "E07000196": "South Staffordshire",
    "E07000197": "Stafford",
    "E07000198": "Staffordshire Moorlands",
    "E07000199": "Tamworth",
    "E07000200": "Babergh",
    "E07000201": "Forest Heath",
    "E07000202": "Ipswich",
    "E07000203": "Mid Suffolk",
    "E07000204": "St Edmundsbury",
    "E07000205": "Suffolk Coastal",
    "E07000206": "Waveney",
    "E07000207": "Elmbridge",
    "E07000208": "Epsom and Ewell",
    "E07000209": "Guildford",
    "E07000210": "Mole Valley",
    "E07000211": "Reigate and Banstead",
    "E07000212": "Runnymede",
    "E07000213": "Spelthorne",
    "E07000214": "Surrey Heath",
    "E07000215": "Tandridge",
    "E07000216": "Waverley",
    "E07000217": "Woking",
    "E07000218": "North Warwickshire",
    "E07000219": "Nuneaton and Bedworth",
    "E07000220": "Rugby",
    "E07000221": "Stratford-on-Avon",
    "E07000222": "Warwick",
    "E07000223": "Adur",
    "E07000224": "Arun",
    "E07000225": "Chichester",
    "E07000226": "Crawley",
    "E07000227": "Horsham",
    "E07000228": "Mid Sussex",
    "E07000229": "Worthing",
    "E07000234": "Bromsgrove",
    "E07000235": "Malvern Hills",
    "E07000236": "Redditch",
    "E07000237": "Worcester",
    "E07000238": "Wychavon",
    "E07000239": "Wyre Forest",
    "E07000240": "St Albans",
    "E07000241": "Welwyn Hatfield",
    "E07000242": "East Hertfordshire",
    "E07000243": "Stevenage",
    "E07000244": "East Suffolk",
    "E07000245": "West Suffolk",
    "E08000001": "Bolton",
    "E08000002": "Bury",
    "E08000003": "Manchester",
    "E08000004": "Oldham",
    "E08000005": "Rochdale",
    "E08000006": "Salford",
    "E08000007": "Stockport",
    "E08000008": "Tameside",
    "E08000009": "Trafford",
    "E08000010": "Wigan",
    "E08000011": "Knowsley",
    "E08000012": "Liverpool",
    "E08000013": "St. Helens",
    "E08000014": "Sefton",
    "E08000015": "Wirral",
    "E08000016": "Barnsley",
    "E08000017": "Doncaster",
    "E08000018": "Rotherham",
    "E08000019": "Sheffield",
    "E08000020": "Gateshead",
    "E08000021": "Newcastle upon Tyne",
    "E08000022": "North Tyneside",
    "E08000023": "South Tyneside",
    "E08000024": "Sunderland",
    "E08000025": "Birmingham",
    "E08000026": "Coventry",
    "E08000027": "Dudley",
    "E08000028": "Sandwell",
    "E08000029": "Solihull",
    "E08000030": "Walsall",
    "E08000031": "Wolverhampton",
    "E08000032": "Bradford",
    "E08000033": "Calderdale",
    "E08000034": "Kirklees",
    "E08000035": "Leeds",
    "E08000036": "Wakefield",
    "E09000001": "City of London",
    "E09000002": "Barking and Dagenham",
    "E09000003": "Barnet",
    "E09000004": "Bexley",
    "E09000005": "Brent",
    "E09000006": "Bromley",
    "E09000007": "Camden",
    "E09000008": "Croydon",
    "E09000009": "Ealing",
    "E09000010": "Enfield",
    "E09000011": "Greenwich",
    "E09000012": "Hackney",
    "E09000013": "Hammersmith and Fulham",
    "E09000014": "Haringey",
    "E09000015": "Harrow",
    "E09000016": "Havering",
    "E09000017": "Hillingdon",
    "E09000018": "Hounslow",
    "E09000019": "Islington",
    "E09000020": "Kensington and Chelsea",
      "E09000021": "Kingston upon Thames",
    "E09000022": "Lambeth",
    "E09000023": "Lewisham",
    "E09000024": "Merton",
    "E09000025": "Newham",
    "E09000026": "Redbridge",
    "E09000027": "Richmond upon Thames",
    "E09000028": "Southwark",
    "E09000029": "Sutton",
    "E09000030": "Tower Hamlets",
    "E09000031": "Waltham Forest",
    "E09000032": "Wandsworth",
    "E09000033": "Westminster",
    "EHEATHROW": "London Airport (Heathrow)",
    "S12000005": "Clackmannanshire",
    "S12000006": "Dumfries and Galloway",
    "S12000008": "East Ayrshire",
    "S12000009": "East Dunbartonshire",
    "S12000010": "East Lothian",
    "S12000011": "East Renfrewshire",
    "S12000013": "Comhairle nan Eilean Siar",
    "S12000014": "Falkirk",
    "S12000015": "Fife",
    "S12000017": "Highland",
    "S12000018": "Inverclyde",
    "S12000019": "Midlothian",
    "S12000020": "Moray",
    "S12000021": "North Ayrshire",
    "S12000023": "Orkney Islands",
    "S12000024": "Perth and Kinross",
    "S12000026": "Scottish Borders",
    "S12000027": "Shetland Islands",
    "S12000028": "South Ayrshire",
    "S12000029": "South Lanarkshire",
    "S12000030": "Stirling",
    "S12000033": "Aberdeen City",
    "S12000034": "Aberdeenshire",
    "S12000035": "Argyll and Bute",
    "S12000036": "City of Edinburgh",
    "S12000038": "Renfrewshire",
    "S12000039": "West Dunbartonshire",
    "S12000040": "West Lothian",
    "S12000041": "Angus",
    "S12000042": "Dundee City",
    "S12000043": "Glasgow City",
    "S12000044": "North Lanarkshire",
    "S12000045": "East Dunbartonshire",
    "S12000046": "Glasgow City",
    "W06000001": "Isle of Anglesey",
    "W06000002": "Gwynedd",
    "W06000003": "Conwy",
    "W06000004": "Denbighshire",
    "W06000005": "Flintshire",
    "W06000006": "Wrexham",
    "W06000008": "Ceredigion",
    "W06000009": "Pembrokeshire",
    "W06000010": "Carmarthenshire",
    "W06000011": "Swansea",
    "W06000012": "Neath Port Talbot",
    "W06000013": "Bridgend",
    "W06000014": "Vale of Glamorgan",
    "W06000015": "Cardiff",
    "W06000016": "Rhondda Cynon Taf",
    "W06000018": "Caerphilly",
    "W06000019": "Blaenau Gwent",
    "W06000020": "Torfaen",
    "W06000021": "Monmouthshire",
    "W06000022": "Newport",
    "W06000023": "Powys",
    "W06000024": "Merthyr Tydfil"
}, inplace=True)


In [95]:
df["local_authority_highway"].replace({
    "E06000001": "Hartlepool",
    "E06000002": "Middlesbrough",
    "E06000003": "Redcar and Cleveland",
    "E06000004": "Stockton-on-Tees",
    "E06000005": "Darlington",
    "E06000006": "Halton",
    "E06000007": "Warrington",
    "E06000008": "Blackburn with Darwen",
    "E06000009": "Blackpool",
    "E06000010": "Kingston upon Hull, City of",
    "E06000011": "East Riding of Yorkshire",
    "E06000012": "North East Lincolnshire",
    "E06000013": "North Lincolnshire",
    "E06000014": "York",
    "E06000015": "Derby",
    "E06000016": "Leicester",
    "E06000017": "Rutland",
    "E06000018": "Nottingham",
    "E06000019": "Herefordshire, County of",
    "E06000020": "Telford and Wrekin",
    "E06000021": "Stoke-on-Trent",
    "E06000022": "Bath and North East Somerset",
    "E06000023": "Bristol, City of",
    "E06000024": "North Somerset",
    "E06000025": "South Gloucestershire",
    "E06000026": "Plymouth",
    "E06000027": "Torbay",
    "E06000028": "Bournemouth",
    "E06000029": "Poole",
    "E06000030": "Swindon",
    "E06000031": "Peterborough",
    "E06000032": "Luton",
    "E06000033": "Southend-on-Sea",
    "E06000034": "Thurrock",
    "E06000035": "Medway",
    "E06000036": "Bracknell Forest",
    "E06000037": "West Berkshire",
    "E06000038": "Reading",
    "E06000039": "Slough",
    "E06000040": "Windsor and Maidenhead",
    "E06000041": "Wokingham",
    "E06000042": "Milton Keynes",
    "E06000043": "Brighton and Hove",
    "E06000044": "Portsmouth",
    "E06000045": "Southampton",
    "E06000046": "Isle of Wight",
    "E06000047": "County Durham",
    "E06000048": "Northumberland",
    "E06000049": "Cheshire East",
    "E06000050": "Cheshire West and Chester",
    "E06000051": "Shropshire",
    "E06000052": "Cornwall",
    "E06000053": "Isles of Scilly",
    "E06000054": "Wiltshire",
    "E06000055": "Bedford",
    "E06000056": "Central Bedfordshire",
    "E08000001": "Bolton",
    "E08000002": "Bury",
    "E08000003": "Manchester",
    "E08000004": "Oldham",
    "E08000005": "Rochdale",
    "E08000006": "Salford",
    "E08000007": "Stockport",
    "E08000008": "Tameside",
    "E08000009": "Trafford",
    "E08000010": "Wigan",
    "E08000011": "Knowsley",
    "E08000012": "Liverpool",
    "E08000013": "St. Helens",
    "E08000014": "Sefton",
    "E08000015": "Wirral",
    "E08000016": "Barnsley",
    "E08000017": "Doncaster",
    "E08000018": "Rotherham",
    "E08000019": "Sheffield",
    "E08000020": "Gateshead",
    "E08000021": "Newcastle upon Tyne",
    "E08000022": "North Tyneside",
    "E08000023": "South Tyneside",
    "E08000024": "Sunderland",
    "E08000025": "Birmingham",
    "E08000026": "Coventry",
    "E08000027": "Dudley",
    "E08000028": "Sandwell",
    "E08000029": "Solihull",
    "E08000030": "Walsall",
    "E08000031": "Wolverhampton",
    "E08000032": "Bradford",
    "E08000033": "Calderdale",
    "E08000034": "Kirklees",
    "E08000035": "Leeds",
    "E08000036": "Wakefield",
    "E09000001": "City of London",
    "E09000002": "Barking and Dagenham",
    "E09000003": "Barnet",
    "E09000004": "Bexley",
    "E09000005": "Brent",
    "E09000006": "Bromley",
    "E09000007": "Camden",
     "E09000008": "Croydon",
    "E09000009": "Ealing",
    "E09000010": "Enfield",
    "E09000011": "Greenwich",
    "E09000012": "Hackney",
    "E09000013": "Hammersmith and Fulham",
    "E09000014": "Haringey",
    "E09000015": "Harrow",
    "E09000016": "Havering",
    "E09000017": "Hillingdon",
    "E09000018": "Hounslow",
    "E09000019": "Islington",
    "E09000020": "Kensington and Chelsea",
    "E09000021": "Kingston upon Thames",
    "E09000022": "Lambeth",
    "E09000023": "Lewisham",
    "E09000024": "Merton",
    "E09000025": "Newham",
    "E09000026": "Redbridge",
    "E09000027": "Richmond upon Thames",
    "E09000028": "Southwark",
    "E09000029": "Sutton",
    "E09000030": "Tower Hamlets",
    "E09000031": "Waltham Forest",
    "E09000032": "Wandsworth",
    "E09000033": "Westminster",
    "E10000002": "Buckinghamshire",
    "E10000003": "Cambridgeshire",
    "E10000006": "Cumbria",
    "E10000007": "Derbyshire",
    "E10000008": "Devon",
    "E10000009": "Dorset",
    "E10000011": "East Sussex",
    "E10000012": "Essex",
    "E10000013": "Gloucestershire",
    "E10000014": "Hampshire",
    "E10000015": "Hertfordshire",
    "E10000016": "Kent",
    "E10000017": "Lancashire",
    "E10000018": "Leicestershire",
    "E10000019": "Lincolnshire",
    "E10000020": "Norfolk",
    "E10000021": "Northamptonshire",
    "E10000023": "North Yorkshire",
    "E10000024": "Nottinghamshire",
    "E10000025": "Oxfordshire",
    "E10000027": "Somerset",
    "E10000028": "Staffordshire",
    "E10000029": "Suffolk",
    "E10000030": "Surrey",
    "E10000031": "Warwickshire",
    "E10000032": "West Sussex",
    "E10000034": "Worcestershire",
    "EHEATHROW": "London Airport (Heathrow)",
    "S12000005": "Clackmannanshire",
    "S12000006": "Dumfries & Galloway",
    "S12000008": "East Ayrshire",
    "S12000009": "East Dunbartonshire",
    "S12000010": "East Lothian",
    "S12000011": "East Renfrewshire",
    "S12000013": "Na h-Eileanan an Iar (Western Isles)",
    "S12000014": "Falkirk",
    "S12000015": "Fife",
    "S12000017": "Highland",
    "S12000018": "Inverclyde",
    "S12000019": "Midlothian",
    "S12000020": "Moray",
    "S12000021": "North Ayrshire",
    "S12000023": "Orkney Islands",
    "S12000024": "Perth and Kinross",
    "S12000026": "Scottish Borders",
    "S12000027": "Shetland Islands",
    "S12000028": "South Ayrshire",
    "S12000029": "South Lanarkshire",
    "S12000030": "Stirling",
    "S12000033": "Aberdeen City",
    "S12000034": "Aberdeenshire",
    "S12000035": "Argyll & Bute",
    "S12000036": "Edinburgh, City of",
    "S12000038": "Renfrewshire",
    "S12000039": "West Dunbartonshire",
    "S12000040": "West Lothian",
    "S12000041": "Angus",
    "S12000042": "Dundee City",
    "S12000043": "Glasgow City",
    "S12000044": "North Lanarkshire",
    "W06000001": "Isle of Anglesey",
    "W06000002": "Gwynedd",
    "W06000003": "Conwy",
    "W06000004": "Denbighshire",
    "W06000005": "Flintshire",
    "W06000006": "Wrexham",
    "W06000008": "Ceredigion",
    "W06000009": "Pembrokeshire",
    "W06000010": "Carmarthenshire",
    "W06000011": "Swansea",
    "W06000012": "Neath Port Talbot",
    "W06000013": "Bridgend",
    "W06000014": "The Vale of Glamorgan",
    "W06000015": "Cardiff",
    "W06000016": "Rhondda, Cynon, Taff",
    "W06000018": "Caerphilly",
    "W06000019": "Blaenau Gwent",
    "W06000020": "Torfaen",
    "W06000021": "Monmouthshire",
    "W06000022": "Newport",
    "W06000023": "Powys",
    "W06000024": "Merthyr Tydfil"
}, inplace=True)

In [96]:
df["first_road_class"].replace({
    1: 'Motorway',
    2: 'A(M)',
    3: 'A',
    4: 'B',
    5: 'C',
    6: 'Unclassified'
}, inplace=True)


In [97]:
df["first_road_number"].replace({
    -1: 'Unknown',
    0: 'first_road_class is C or Unclassified. These roads do not have official numbers so recorded as zero'
}, inplace=True)


In [98]:
df["road_type"].replace({
    1: 'Roundabout',
    2: 'One way street',
    3: 'Dual carriageway',
    6: 'Single carriageway',
    7: 'Slip road',
    9: 'Unknown',
    12: 'One way street/Slip road',
    -1: 'Data missing or out of range'
}, inplace=True)


In [99]:
df["speed_limit"].replace({
    -1: 'Data missing or out of range',
    99: 'unknown (self reported)'
}, inplace=True)


In [100]:
df["junction_detail"].replace({
    0: 'Not at junction or within 20 metres',
    1: 'Roundabout',
    2: 'Mini-roundabout',
    3: 'T or staggered junction',
    5: 'Slip road',
    6: 'Crossroads',
    7: 'More than 4 arms (not roundabout)',
    8: 'Private drive or entrance',
    9: 'Other junction',
    99: 'unknown (self reported)',
    -1: 'Data missing or out of range'
}, inplace=True)


In [101]:
df["junction_control"].replace({
    0: 'Not at junction or within 20 metres',
    1: 'Authorised person',
    2: 'Auto traffic signal',
    3: 'Stop sign',
    4: 'Give way or uncontrolled',
    -1: 'Data missing or out of range',
    9: 'unknown (self reported)'
}, inplace=True)


In [102]:
df["second_road_class"].replace({
    0: 'Not at junction or within 20 metres',
    1: 'Motorway',
    2: 'A(M)',
    3: 'A',
    4: 'B',
    5: 'C',
    6: 'Unclassified'
}, inplace=True)


In [103]:
df["second_road_number"].replace({
    -1: 'Unknown',
    0: 'first_road_class is C or Unclassified. These roads do not have official numbers so recorded as zero'
}, inplace=True)


In [104]:
df["pedestrian_crossing_human_control"].replace({
    0: 'None within 50 metres',
    1: 'Control by school crossing patrol',
    2: 'Control by other authorised person',
    -1: 'Data missing or out of range',
    9: 'Unknown (self reported)'
}, inplace=True)


In [105]:
df["pedestrian_crossing_physical_facilities"].replace({
    0: 'No physical crossing facilities within 50 metres',
    1: 'Zebra',
    4: 'Pelican, puffin, toucan or similar non-junction pedestrian light crossing',
    5: 'Pedestrian phase at traffic signal junction',
    7: 'Footbridge or subway',
    8: 'Central refuge',
    -1: 'Data missing or out of range',
    9: 'Unknown (self reported)'
}, inplace=True)


In [155]:
df[df['weather_conditions'] == 'Fine + high winds']

,accident_index,accident_year_x,accident_reference_x,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,accident_year_y,accident_reference_y,vehicle_reference_x,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,lsoa_of_driver,accident_year,accident_reference,vehicle_reference_y,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile,lsoa_of_casualty
351,2022010353008,2022,10353008,526252.0,182619.0,-0.181388,51.528378,Metropolitan Police,Slight,2,2,04/01/2022,Tuesday,19:30,-1,Westminster,Westminster,A,5,Single carriageway,20,Crossroads,Auto traffic signal,C,first_road_class is C or Unclassified. These r...,None within 50 metres,Pedestrian phase at traffic signal junction,Darkness - lights lit,Fine + high winds,Dry,None,None,Urban,Yes,Non-trunk,E01004726,2022.0,10353008,1.0,Motorcycle 125cc and under,No tow/articulation,Going ahead other,West,East,On main carriageway - not in restricted lane,Mid Junction - on roundabout or on main road,Skidded,None,Did not leave carriageway,None,Front,No,Not known,Male,36.0,36 - 45,125.0,Petrol,2.0,YAMAHA GPD,More deprived 20-30%,Urban area,E01000642,2022.0,10353008,1.0,1.0,Driver or rider,Male,36.0,36 - 45,Slight,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Motorcycle 125cc and under rider or passenger,Urban area,More deprived 20-30%,E01000642
352,2022010353008,2022,10353008,526252.0,182619.0,-0.181388,51.528378,Metropolitan Police,Slight,2,2,04/01/2022,Tuesday,19:30,-1,Westminster,Westminster,A,5,Single carriageway,20,Crossroads,Auto traffic signal,C,first_road_class is C or Unclassified. These r...,None within 50 metres,Pedestrian phase at traffic signal junction,Darkness - lights lit,Fine + high winds,Dry,None,None,Urban,Yes,Non-trunk,E01004726,2022.0,10353008,1.0,Motorcycle 125cc and under,No tow/articulation,Going ahead other,West,East,On main carriageway - not in restricted lane,Mid Junction - on roundabout or on main road,Skidded,None,Did not leave carriageway,None,Front,No,Not known,Male,36.0,36 - 45,125.0,Petrol,2.0,YAMAHA GPD,More deprived 20-30%,Urban area,E01000642,2022.0,10353008,2.0,2.0,Driver or rider,Male,26.0,26 - 35,Slight,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Cyclist,Data missing or out of range,Data missing or out of range,-1
353,2022010353008,2022,10353008,526252.0,182619.0,-0.181388,51.528378,Metropolitan Police,Slight,2,2,04/01/2022,Tuesday,19:30,-1,Westminster,Westminster,A,5,Single carriageway,20,Crossroads,Auto traffic signal,C,first_road_class is C or Unclassified. These r...,None within 50 metres,Pedestrian phase at traffic signal junction,Darkness - lights lit,Fine + high winds,Dry,None,None,Urban,Yes,Non-trunk,E01004726,

In [106]:
df["light_conditions"].replace({
    1: 'Daylight',
    4: 'Darkness - lights lit',
    5: 'Darkness - lights unlit',
    6: 'Darkness - no lighting',
    7: 'Darkness - lighting unknown',
    -1: 'Data missing or out of range'
}, inplace=True)


In [107]:
df["weather_conditions"].replace({
    1: 'Fine no high winds',
    2: 'Raining no high winds',
    3: 'Snowing no high winds',
    4: 'Fine + high winds',
    5: 'Raining + high winds',
    6: 'Snowing + high winds',
    7: 'Fog or mist',
    8: 'Other',
    9: 'Unknown',
    -1: 'Data missing or out of range'
}, inplace=True)


In [108]:
df["road_surface_conditions"].replace({
    1: 'Dry',
    2: 'Wet or damp',
    3: 'Snow',
    4: 'Frost or ice',
    5: 'Flood over 3cm deep',
    6: 'Oil or diesel',
    7: 'Mud',
    -1: 'Data missing or out of range',
    9: 'Unknown (self-reported)'
}, inplace=True)


In [109]:
df["special_conditions_at_site"].replace({
    0: 'None',
    1: 'Auto traffic signal - out',
    2: 'Auto signal part defective',
    3: 'Road sign or marking defective or obscured',
    4: 'Roadworks',
    5: 'Road surface defective',
    6: 'Oil or diesel',
    7: 'Mud',
    -1: 'Data missing or out of range',
    9: 'Unknown (self-reported)'
}, inplace=True)


In [110]:
df["carriageway_hazards"].replace({
    0: 'None',
    1: 'Vehicle load on road',
    2: 'Other object on road',
    3: 'Previous accident',
    4: 'Dog on road',
    5: 'Other animal on road',
    6: 'Pedestrian in carriageway - not injured',
    7: 'Any animal in carriageway (except ridden horse)',
    -1: 'Data missing or out of range',
    9: 'Unknown (self-reported)'
}, inplace=True)


In [111]:
df["urban_or_rural_area"].replace({
    1: 'Urban',
    2: 'Rural',
    3: 'Unallocated',
    -1: 'Data missing or out of range'
}, inplace=True)


In [112]:
df["did_police_officer_attend_scene_of_accident"].replace({
    1: 'Yes',
    2: 'No',
    3: 'No - accident was reported using a self completion form (self rep only)',
    -1: 'Data missing or out of range'
}, inplace=True)


In [113]:
df["trunk_road_flag"].replace({
    1: 'Trunk (Roads managed by Highways England)',
    2: 'Non-trunk',
    -1: 'Data missing or out of range'
}, inplace=True)


In [114]:
df["vehicle_type"].replace({
    1: 'Pedal cycle',
    2: 'Motorcycle 50cc and under',
    3: 'Motorcycle 125cc and under',
    4: 'Motorcycle over 125cc and up to 500cc',
    5: 'Motorcycle over 500cc',
    8: 'Taxi/Private hire car',
    9: 'Car',
    10: 'Minibus (8 - 16 passenger seats)',
    11: 'Bus or coach (17 or more pass seats)',
    16: 'Ridden horse',
    17: 'Agricultural vehicle',
    18: 'Tram',
    19: 'Van / Goods 3.5 tonnes mgw or under',
    20: 'Goods over 3.5t. and under 7.5t',
    21: 'Goods 7.5 tonnes mgw and over',
    22: 'Mobility scooter',
    23: 'Electric motorcycle',
    90: 'Other vehicle',
    97: 'Motorcycle - unknown cc',
    98: 'Goods vehicle - unknown weight',
    99: 'Unknown vehicle type (self rep only)',
    103: 'Motorcycle - Scooter (1979-1998)',
    104: 'Motorcycle (1979-1998)',
    105: 'Motorcycle - Combination (1979-1998)',
    106: 'Motorcycle over 125cc (1999-2004)',
    108: 'Taxi (excluding private hire cars) (1979-2004)',
    109: 'Car (including private hire cars) (1979-2004)',
    110: 'Minibus/Motor caravan (1979-1998)',
    113: 'Goods over 3.5 tonnes (1979-1998)',
    -1: 'Data missing or out of range'
}, inplace=True)


In [115]:
df["towing_and_articulation"].replace({
    0: 'No tow/articulation',
    1: 'Articulated vehicle',
    2: 'Double or multiple trailer',
    3: 'Caravan',
    4: 'Single trailer',
    5: 'Other tow',
    9: 'Unknown (self reported)',
    -1: 'Data missing or out of range'
}, inplace=True)


In [116]:
df["vehicle_manoeuvre"].replace({
    1: 'Reversing',
    2: 'Parked',
    3: 'Waiting to go - held up',
    4: 'Slowing or stopping',
    5: 'Moving off',
    6: 'U-turn',
    7: 'Turning left',
    8: 'Waiting to turn left',
    9: 'Turning right',
    10: 'Waiting to turn right',
    11: 'Changing lane to left',
    12: 'Changing lane to right',
    13: 'Overtaking moving vehicle - offside',
    14: 'Overtaking static vehicle - offside',
    15: 'Overtaking - nearside',
    16: 'Going ahead left-hand bend',
    17: 'Going ahead right-hand bend',
    18: 'Going ahead other',
    99: 'Unknown (self reported)',
    -1: 'Data missing or out of range'
}, inplace=True)


In [117]:
df["vehicle_direction_from"].replace({
    0: 'Parked',
    1: 'North',
    2: 'North East',
    3: 'East',
    4: 'South East',
    5: 'South',
    6: 'South West',
    7: 'West',
    8: 'North West',
    9: 'Unknown (self reported)'
}, inplace=True)


In [118]:
df["vehicle_direction_to"].replace({
    0: 'Parked',
    1: 'North',
    2: 'North East',
    3: 'East',
    4: 'South East',
    5: 'South',
    6: 'South West',
    7: 'West',
    8: 'North West',
    9: 'Unknown (self reported)'
}, inplace=True)


In [119]:
df["vehicle_location_restricted_lane"].replace({
    0: 'On main carriageway - not in restricted lane',
    1: 'Tram/Light rail track',
    2: 'Bus lane',
    3: 'Busway (including guided busway)',
    4: 'Cycle lane (on main carriageway)',
    5: 'Cycleway or shared use footway (not part of main carriageway)',
    6: 'On lay-by or hard shoulder',
    7: 'Entering lay-by or hard shoulder',
    8: 'Leaving lay-by or hard shoulder',
    9: 'Footway (pavement)',
    10: 'Not on carriageway',
    99: 'Unknown (self reported)'
}, inplace=True)


In [120]:
df["junction_location"].replace({
    -1: 'Data missing or out of range',
    0: 'Not at or within 20 metres of junction',
    1: 'Approaching junction or waiting/parked at junction approach',
    2: 'Cleared junction or waiting/parked at junction exit',
    3: 'Leaving roundabout',
    4: 'Entering roundabout',
    5: 'Leaving main road',
    6: 'Entering main road',
    7: 'Entering from slip road',
    8: 'Mid Junction - on roundabout or on main road',
    9: 'Unknown (self reported)'
}, inplace=True)


In [121]:
df["skidding_and_overturning"].replace({
    -1: 'Data missing or out of range',
    0: 'None',
    1: 'Skidded',
    2: 'Skidded and overturned',
    3: 'Jackknifed',
    4: 'Jackknifed and overturned',
    5: 'Overturned',
    9: 'Unknown (self reported)'
}, inplace=True)


In [5]:
df["hit_object_in_carriageway"].replace({
    -1: 'Data missing or out of range',
    0: 'None',
    1: 'Previous accident',
    2: 'Road works',
    4: 'Parked vehicle',
    5: 'Bridge (roof)',
    6: 'Bridge (side)',
    7: 'Bollard or refuge',
    8: 'Open door of vehicle',
    9: 'Central island of roundabout',
    10: 'Kerb',
    11: 'Other object',
    12: 'Any animal (except ridden horse)',
    99: 'Unknown (self reported)'
}, inplace=True)


In [9]:
print(df[df['hit_object_in_carriageway'] == 'Kerb'])

       accident_index  accident_year_x accident_reference_x  \
22      2022010352588             2022             10352588   
23      2022010352588             2022             10352588   
24      2022010352588             2022             10352588   
25      2022010352588             2022             10352588   
26      2022010352588             2022             10352588   
...               ...              ...                  ...   
222655  2022991249624             2022            991249624   
222702  2022991250569             2022            991250569   
223026  2022991257052             2022            991257052   
223144  2022991259274             2022            991259274   
223157  2022991260785             2022            991260785   

        location_easting_osgr  location_northing_osgr  longitude   latitude  \
22                   543159.0                181261.0   0.061626  51.512146   
23                   543159.0                181261.0   0.061626  51.512146   
24    

In [123]:
df["vehicle_leaving_carriageway"].replace({
    -1: 'Data missing or out of range',
    0: 'Did not leave carriageway',
    1: 'Nearside',
    2: 'Nearside and rebounded',
    3: 'Straight ahead at junction',
    4: 'Offside on to central reservation',
    5: 'Offside on to central reservation + rebounded',
    6: 'Offside - crossed central reservation',
    7: 'Offside',
    8: 'Offside and rebounded',
    9: 'Unknown (self reported)'
}, inplace=True)


In [124]:
df["hit_object_off_carriageway"].replace({
    -1: 'Data missing or out of range',
    0: 'None',
    1: 'Road sign or traffic signal',
    2: 'Lamp post',
    3: 'Telegraph or electricity pole',
    4: 'Tree',
    5: 'Bus stop or bus shelter',
    6: 'Central crash barrier',
    7: 'Near/Offside crash barrier',
    8: 'Submerged in water',
    9: 'Entered ditch',
    10: 'Other permanent object',
    11: 'Wall or fence',
    99: 'Unknown (self reported)'
}, inplace=True)


In [125]:
df["first_point_of_impact"].replace({
    -1: 'Data missing or out of range',
    0: 'Did not impact',
    1: 'Front',
    2: 'Back',
    3: 'Offside',
    4: 'Nearside',
    9: 'Unknown (self reported)'
}, inplace=True)


In [126]:
df["vehicle_left_hand_drive"].replace({
    -1: 'Data missing or out of range',
    1: 'No',
    2: 'Yes',
    9: 'Unknown'
}, inplace=True)


In [127]:
df["journey_purpose_of_driver"].replace({
    -1: 'Data missing or out of range',
    1: 'Journey as part of work',
    2: 'Commuting to/from work',
    3: 'Taking pupil to/from school',
    4: 'Pupil riding to/from school',
    5: 'Other',
    6: 'Not known',
    15: 'Other/Not known'
}, inplace=True)


In [128]:
df["sex_of_driver"].replace({
    -1: 'Data missing or out of range',
    1: 'Male',
    2: 'Female',
    3: 'Not known'
}, inplace=True)


In [129]:
df["age_of_driver"].replace({
    -1: 'Data missing or out of range'
}, inplace=True)


In [130]:
df["age_band_of_driver"].replace({
    1: '0 - 5',
    2: '6 - 10',
    3: '11 - 15',
    4: '16 - 20',
    5: '21 - 25',
    6: '26 - 35',
    7: '36 - 45',
    8: '46 - 55',
    9: '56 - 65',
    10: '66 - 75',
    11: 'Over 75',
    -1: 'Data missing or out of range'
}, inplace=True)


In [131]:
df["engine_capacity_cc"].replace({
    -1: 'Data missing or out of range'
}, inplace=True)


In [132]:
df["propulsion_code"].replace({
    1: 'Petrol',
    2: 'Heavy oil',
    3: 'Electric',
    4: 'Steam',
    5: 'Gas',
    6: 'Petrol/Gas (LPG)',
    7: 'Gas/Bi-fuel',
    8: 'Hybrid electric',
    9: 'Gas Diesel',
    10: 'New fuel technology',
    11: 'Fuel cells',
    12: 'Electric diesel',
    -1: 'Undefined'
}, inplace=True)


In [133]:
df["generic_make_model"].replace({
    -1: 'Data missing or out of range'
}, inplace=True)


In [134]:
df["driver_imd_decile"].replace({
    -1: 'Data missing or out of range'
}, inplace=True)


In [135]:
df["driver_imd_decile"].replace({
    1: 'Most deprived 10%',
    2: 'More deprived 10-20%',
    3: 'More deprived 20-30%',
    4: 'More deprived 30-40%',
    5: 'More deprived 40-50%',
    6: 'Less deprived 40-50%',
    7: 'Less deprived 30-40%',
    8: 'Less deprived 20-30%',
    9: 'Less deprived 10-20%',
    10: 'Least deprived 10%',
    -1: 'Data missing or out of range'
}, inplace=True)


In [136]:
df["driver_home_area_type"].replace({
    1: 'Urban area',
    2: 'Small town',
    3: 'Rural',
    -1: 'Data missing or out of range'
}, inplace=True)


In [137]:
df["casualty_class"].replace({
   1: 'Driver or rider',
   2: 'Passenger',
   3:'Pedestrian', 
}, inplace=True)


In [138]:
df["sex_of_casualty"].replace({
    1: 'Male',
    2: 'Female',
    9: 'unknown (self reported)',
    -1: 'Data missing or out of range'
}, inplace=True)


In [139]:
df["age_of_casualty"].replace({
    -1: 'Data missing or out of range'
}, inplace=True)


In [140]:
df["age_band_of_casualty"].replace({
    1: '0 - 5',
    2: '6 - 10',
    3: '11 - 15',
    4: '16 - 20',
    5: '21 - 25',
    6: '26 - 35',
    7: '36 - 45',
    8: '46 - 55',
    9: '56 - 65',
    10: '66 - 75',
    11: 'Over 75',
    -1: 'Data missing or out of range'
}, inplace=True)


In [141]:
df["casualty_severity"].replace({
    1: 'Fatal',
    2: 'Serious',
    3: 'Slight'
}, inplace=True)


In [142]:
df["pedestrian_location"].replace({
    0: 'Not a Pedestrian',
    1: 'Crossing on pedestrian crossing facility',
    2: 'Crossing in zig-zag approach lines',
    3: 'Crossing in zig-zag exit lines',
    4: 'Crossing elsewhere within 50m. of pedestrian crossing',
    5: 'In carriageway, crossing elsewhere',
    6: 'On footway or verge',
    7: 'On refuge, central island or central reservation',
    8: 'In centre of carriageway - not on refuge, island or central reservation',
    9: 'In carriageway, not crossing',
    10: 'Unknown or other'
}, inplace=True)


In [143]:
df["pedestrian_location"].replace({
    0: 'Not a Pedestrian',
    1: 'Crossing on pedestrian crossing facility',
    2: 'Crossing in zig-zag approach lines',
    3: 'Crossing in zig-zag exit lines',
    4: 'Crossing elsewhere within 50m. of pedestrian crossing',
    5: 'In carriageway, crossing elsewhere',
    6: 'On footway or verge',
    7: 'On refuge, central island or central reservation',
    8: 'In centre of carriageway - not on refuge, island or central reservation',
    9: 'In carriageway, not crossing',
    10: 'Unknown or other',
    -1: 'Data missing or out of range'
}, inplace=True)


In [144]:
df["pedestrian_movement"].replace({
    0: 'Not a Pedestrian',
    1: "Crossing from driver's nearside",
    2: 'Crossing from nearside - masked by parked or stationary vehicle',
    3: "Crossing from driver's offside",
    4: 'Crossing from offside - masked by parked or stationary vehicle',
    5: 'In carriageway, stationary - not crossing  (standing or playing)',
    6: 'In carriageway, stationary - not crossing  (standing or playing) - masked by parked or stationary vehicle',
    7: 'Walking along in carriageway, facing traffic',
    8: 'Walking along in carriageway, back to traffic',
    9: 'Unknown or other',
    -1: 'Data missing or out of range'
}, inplace=True)


In [145]:
df["car_passenger"].replace({
    0: 'Not car passenger',
    1: 'Front seat passenger',
    2: 'Rear seat passenger',
    9: 'unknown (self reported)',
    -1: 'Data missing or out of range'
}, inplace=True)


In [146]:
df["bus_or_coach_passenger"].replace({
    0: 'Not a bus or coach passenger',
    1: 'Boarding',
    2: 'Alighting',
    3: 'Standing passenger',
    4: 'Seated passenger',
    9: 'unknown (self reported)',
    -1: 'Data missing or out of range'
}, inplace=True)


In [147]:
df["pedestrian_road_maintenance_worker"].replace({
    0: 'No / Not applicable',
    1: 'Yes',
    2: 'Not Known',
    3: 'Probable',
    -1: 'Data missing or out of range'
}, inplace=True)


In [148]:
df["casualty_type"].replace({
    0: 'Pedestrian',
    1: 'Cyclist',
    2: 'Motorcycle 50cc and under rider or passenger',
    3: 'Motorcycle 125cc and under rider or passenger',
    4: 'Motorcycle over 125cc and up to 500cc rider or passenger',
    5: 'Motorcycle over 500cc rider or passenger',
    8: 'Taxi/Private hire car occupant',
    9: 'Car occupant',
    10: 'Minibus (8 - 16 passenger seats) occupant',
    11: 'Bus or coach occupant (17 or more pass seats)',
    16: 'Horse rider',
    17: 'Agricultural vehicle occupant',
    18: 'Tram occupant',
    19: 'Van / Goods vehicle (3.5 tonnes mgw or under) occupant',
    20: 'Goods vehicle (over 3.5t. and under 7.5t.) occupant',
    21: 'Goods vehicle (7.5 tonnes mgw and over) occupant',
    22: 'Mobility scooter rider',
    23: 'Electric motorcycle rider or passenger',
    90: 'Other vehicle occupant',
    97: 'Motorcycle - unknown cc rider or passenger',
    98: 'Goods vehicle (unknown weight) occupant',
    99: 'Unknown vehicle type (self rep only)',
    103: 'Motorcycle - Scooter (1979-1998)',
    104: 'Motorcycle (1979-1998)',
    105: 'Motorcycle - Combination (1979-1998)',
    106: 'Motorcycle over 125cc (1999-2004)',
    108: 'Taxi (excluding private hire cars) (1979-2004)',
    109: 'Car (including private hire cars) (1979-2004)',
    110: 'Minibus/Motor caravan (1979-1998)',
    113: 'Goods over 3.5 tonnes (1979-1998)'
}, inplace=True)


In [149]:
df["casualty_imd_decile"].replace({
    1: 'Most deprived 10%',
    2: 'More deprived 10-20%',
    3: 'More deprived 20-30%',
    4: 'More deprived 30-40%',
    5: 'More deprived 40-50%',
    6: 'Less deprived 40-50%',
    7: 'Less deprived 30-40%',
    8: 'Less deprived 20-30%',
    9: 'Less deprived 10-20%',
    10: 'Least deprived 10%',
    -1: 'Data missing or out of range'
}, inplace=True)


In [150]:
df["casualty_home_area_type"].replace({
    1: 'Urban area',
    2: 'Small town',
    3: 'Rural',
    -1: 'Data missing or out of range'
}, inplace=True)


In [151]:
df

,accident_index,accident_year_x,accident_reference_x,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,accident_year_y,accident_reference_y,vehicle_reference_x,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,lsoa_of_driver,accident_year,accident_reference,vehicle_reference_y,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile,lsoa_of_casualty
0,2022010352073,2022,10352073,525199.0,177928.0,-0.198224,51.486454,Metropolitan Police,Slight,2,1,05/01/2022,Wednesday,16:40,-1,Hammersmith and Fulham,Hammersmith and Fulham,A,3218,Single carriageway,30,Crossroads,Give way or uncontrolled,Unclassified,first_road_class is C or Unclassified. These r...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk,E01001883,2022.0,10352073,1.0,Van / Goods 3.5 tonnes mgw or under,No tow/articulation,Turning right,East,North West,On main carriageway - not in restricted lane,Approaching junction or waiting/parked at junc...,None,None,Did not leave carriageway,None,Front,Yes,Commuting to/from work,Male,48.0,46 - 55,1461.0,Heavy oil,4.0,RENAULT KANGOO,Less deprived 30-40%,Urban area,E01002687,2022.0,10352073,2.0,1.0,Driver or rider,Male,17.0,16 - 20,Slight,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Motorcycle 125cc and under rider or passenger,Urban area,Most deprived 10%,E01001364
1,2022010352073,2022,10352073,525199.0,177928.0,-0.198224,51.486454,Metropolitan Police,Slight,2,1,05/01/2022,Wednesday,16:40,-1,Hammersmith and Fulham,Hammersmith and Fulham,A,3218,Single carriageway,30,Crossroads,Give way or uncontrolled,Unclassified,first_road_class is C or Unclassified. These r...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,Yes,Non-trunk,E01001883,2022.0,10352073,2.0,Motorcycle 125cc and under,No tow/articulation,Going ahead other,North,South,On main carriageway - not in restricted lane,Approaching junction or waiting/parked at junc...,None,None,Did not leave carriageway,None,Front,No,Commuting to/from work,Male,17.0,16 - 20,125.0,Petrol,1.0,HONDA FSH125,Most deprived 10%,Urban area,E01001364,2022.0,10352073,2.0,1.0,Driver or rider,Male,17.0,16 - 20,Slight,Not a Pedestrian,Not a Pedestrian,Not car passenger,Not a bus or coach passenger,No / Not applicable,Motorcycle 125cc and under rider or passenger,Urban area,Most deprived 10%,E01001364
2,2022010352573,2022,10352573,546214.0,179866.0,0.105042,51.498830,Metropolitan Police,Slight,2,1,01/01/2022,Saturday,01:17,-1,Greenwich,Greenwich,A,2016,Dual carriageway,50,Crossroads,Auto traffic signal,Unclassified,first_road_class is C or Unclassifie

In [152]:
df.to_csv('merged_collision_data.csv', index=False)